In [1]:
!pip install pytorch_lightning
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 31.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import torch.utils.data as data
import zipfile
import wandb


# Mount Google Drive to access the dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Set the path to your zipped dataset
zip_file_path = '/content/drive/MyDrive/M.Tech IITM/Deep Learning /aksharantar_sampled.zip'

# Extract the dataset to a folder in Google Drive
zip_ref = zipfile.ZipFile(zip_file_path, 'r')
zip_ref.extractall('aksharantar_sampled_extracted')
zip_ref.close()

In [4]:
import torch
import torch.nn as nn
from google.colab import drive
import csv
# from pytorch_lightning.loggers import WandbLogger
# move tensors to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



# Path to your CSV file on Google Drive (Extracted file)
csv_file_path = '/content/aksharantar_sampled_extracted/aksharantar_sampled/hin/hin_train.csv'

# Read the CSV file and extract the sequence of characters
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []
    for row in reader:
        chars.extend(row[0])  # assuming the first column of the CSV file contains the text data

charS=set(chars)
charS.add('|')
char_set = list(charS)


# Define the mapping between characters and integer indices
char_to_idx_latin= {char: i+1 for i, char in enumerate(char_set)}


In [5]:
max_length_devanagari=0
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []

    for row in reader:
        chars.extend(row[1])  # assuming the first column of the CSV file contains the text data

# Define the character set
charS=set(chars)
charS.add('|')
char_set = list(charS)

# Define the mapping between characters and integer indices
char_to_idx_lang ={char: i+1 for i, char in enumerate(char_set)}

In [6]:

max_length_latin = 0

# Read the CSV file and extract the sequence of characters
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []

    for row in reader:
        length = len(row[0])  # assuming the column you want to check is the first one
        if length > max_length_latin:
            max_length_latin = length

In [7]:
# max_length_latin

In [8]:
max_length_devanagari = 0
# Read the CSV file and extract the sequence of characters
with open(csv_file_path, 'r') as f:
    reader = csv.reader(f)
    chars = []

    for row in reader:
        length = len(row[1])  # assuming the column you want to check is the first one
        if length > max_length_devanagari:
            max_length_devanagari = length

In [9]:
# char_to_idx_lang=char_to_idx_lang.to(device)
# char_to_idx_latin=char_to_idx_latin.to(device)

In [10]:
# char_set

In [11]:
# char_to_idx_latin

In [12]:
# max_length_devanagari

In [13]:
def word_to_indices(word, max_length,dict):
    # Convert the characters to integer indices using the char_to_idx mapping
    indices = [dict.get(c, -1) for c in word]
    # Filter out characters not in the dictionary
    indices = [idx for idx in indices if idx >= 0]
    # Add padding if necessary to make the sequence length equal to max_length
    if len(indices) < max_length:
        indices += [0] * (max_length - len(indices))
    # Truncate the sequence if necessary to make the sequence length equal to max_length
    elif len(indices) > max_length:
        indices = indices[:max_length]
    # Add the start and end tokens to the sequence
    indices = [dict['|']] + indices + [dict['|']]
    # Convert the list of indices to a tensor
    tensor = torch.tensor(indices)
    tensor=tensor.to(device)
    return tensor
pairs=[]

# Read the CSV file containing Latin-Devanagari word pairs
with open('/content/aksharantar_sampled_extracted/aksharantar_sampled/hin/hin_train.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        latin_word = row[0]
        devanagari_word = row[1]
        # Convert the Latin word to a tensor of integer indices
        latin_indices = word_to_indices(latin_word, max_length_latin,char_to_idx_latin)
        devanagari_indices= word_to_indices(devanagari_word,max_length_devanagari ,char_to_idx_lang)
        pairs.append([latin_indices,devanagari_indices])
        # print(devanagari_word, devanagari_indices)
        # print(latin_word,latin_indices)
        # print('**************************************************')

pairs_v=[]

# Read the CSV file containing Latin-Devanagari word pairs
with open('/content/aksharantar_sampled_extracted/aksharantar_sampled/hin/hin_valid.csv', 'r') as f_v:
    reader_v = csv.reader(f_v)
    for row in reader_v:
        latin_word = row[0]
        devanagari_word = row[1]
        # Convert the Latin word to a tensor of integer indices
        latin_indices = word_to_indices(latin_word, max_length_latin,char_to_idx_latin)
        devanagari_indices= word_to_indices(devanagari_word,max_length_devanagari ,char_to_idx_lang)
        pairs_v.append([latin_indices,devanagari_indices])
        # print(devanagari_word, devanagari_indices)
        # print(latin_word,latin_indices)
        # print('**************************************************')


In [14]:
# pairs_v

In [15]:
train_dataloader = torch.utils.data.DataLoader(pairs, batch_size=64, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(pairs_v, batch_size=64, shuffle=False)
# wandb_logger = WandbLogger(project="Assignment 3 Q1 a")
# a2e6402ce9fe2ebe1f01d5332c4fafa210b0dc0c


In [18]:
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, embedding_size,cell_type,drop_out,num_layers,bidirectional):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(input_dim, embedding_size)
        if(cell_type=="GRU"):
          self.rnn = nn.GRU(embedding_size, hidden_dim,dropout=drop_out,num_layers=num_layers)
        if(cell_type=="LSTM"):
          self.rnn = nn.LSTM(hidden_dim, hidden_dim,dropout=drop_out,num_layers=num_layers)
        if(cell_type=="RNN"):
          self.rnn = nn.RNN(hidden_dim, hidden_dim,dropout=drop_out,num_layers=num_layers)

    def forward(self, x):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded)
        return hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim,embedding_size ,cell_type,drop_out,num_layers,bidirectional):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = nn.Embedding(hidden_dim, embedding_size)

        if(cell_type=="GRU"):
          self.rnn = nn.GRU(embedding_size, hidden_dim,dropout=drop_out,num_layers=num_layers)
        elif(cell_type=="LSTM"):
          self.rnn = nn.LSTM(embedding_size, hidden_dim,dropout=drop_out,num_layers=num_layers)
        elif(cell_type=="RNN"):
          self.rnn = nn.RNN(embedding_size, hidden_dim,dropout=drop_out,num_layers=num_layers)

        # self.rnn = nn.GRU(embedding_size, hidden_dim)
        # self.rnn = nn.LSTM(hidden_dim, hidden_dim)
        # self.rnn = nn.RNN(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, hidden):
        # print("hidden",hidden.shape)
        # print("Bef x",x.shape)
        x = x.unsqueeze(0)
        # print("Aft x",x.shape)
        embedded = self.embedding(x)
        # print("embedded",embedded.shape)
        # print("hidden",hidden.shape)
        output, hidden = self.rnn(embedded, hidden)
        prediction = self.fc(output.squeeze(0))
        return prediction, hidden

class Seq2Seq(pl.LightningModule):
    def __init__(self, input_dim, output_dim, hidden_dim,embedding_size, cell_type, drop_out,num_layers_encoders,num_layers_decoders,bidirectional):
        super().__init__()
        self.encoder = Encoder(input_dim, hidden_dim, embedding_size,cell_type,drop_out,num_layers=num_layers_encoders)
        self.decoder = Decoder(output_dim, hidden_dim, embedding_size, cell_type,drop_out,num_layers=num_layers_decoders)
        self.teacher_forcing_ratio = 0.5


    def forward(self, src, trg,teacher_forcing_ratio=0.5):############################################Where r we calling this forward from

        batch_size = trg.shape[0]
        max_len = trg.shape[1]

        trg_vocab_size = self.decoder.fc.out_features
        # print("src",src.shape,"trg",trg.shape)
        src = src.transpose(0,1)
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        hidden = self.encoder(src)
        # print("trg",trg.shape)
        # input = trg[0,:]
        input = trg[:,0]#start character
        # print("INP,Dec",input.shape)
        for t in range(1, max_len):
            # print("t",t,max_len,input.shape)
            output, hidden = self.decoder(input, hidden)
            # print("Out",outputs.shape,output.shape)
            outputs[t] = output
            top1 = output.argmax(1)
            input = top1 if self.teacher_forcing_ratio > torch.rand(1).item() else trg[:,t]
            #if teacher_forcing_ratio > the random number generated then use the predicted character at the timestep t in timestamp t+1, else use the true value from the target 
        return outputs

    def training_step(self, batch, batch_idx):
        src, trg = batch
        # print("Before forward trg",trg.shape)
        output = self(src, trg)
        outputAcc = self(src, trg)
        trgAcc=trg

        outputAcc = outputAcc.permute(1, 0, 2)
    
        cols = torch.arange(output.shape[1]).unsqueeze(1)
        rows = torch.arange(output.shape[0])
        expected = torch.zeros(size=output.shape)
        expected[rows, cols, trg.cpu()] = 1

        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        expected = expected[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = self.loss_fn(output.to(device), expected.to(device))
        # print("loss ",loss)
        self.log("loss", loss, prog_bar=True,on_epoch=True)

        train_accuracy =self.accuracy(outputAcc, trgAcc)    #trg is the true value
        self.log("train_accuracy", train_accuracy, prog_bar=True,on_epoch=True)
        return {'loss': loss}

    def validation_step(self, batch, batch_idx):

        src, trg = batch
        output = self(src, trg,0)
        outputAcc = self(src, trg,0)
        trgAcc=trg

        outputAcc = outputAcc.permute(1, 0, 2)

        cols = torch.arange(output.shape[1]).unsqueeze(1)
        rows = torch.arange(output.shape[0])
        expected = torch.zeros(size=output.shape)
        expected[rows, cols, trg.cpu()] = 1
        # print(output.shape,expected.shape)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        expected = expected[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)


        loss = self.loss_fn(output.to(device), expected.to(device))


        val_accuracy =self.accuracy(outputAcc, trgAcc)    #trg is the true value
        self.log("val_accuracy", val_accuracy, prog_bar=True,on_epoch=True)

        self.log("val loss", loss, prog_bar=True,on_epoch=True)

        return {'loss': loss}


    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters())
        return optimizer

    def loss_fn(self, output, trg):
        # mask = (trg != self.pad_idx).float()
        criterion = nn.CrossEntropyLoss()
        # print("Loss cal",output.shape,trg.shape)
        loss = criterion(output, trg)
        # loss = criterion(output, trg) * mask
        return loss.mean()

    def accuracy(self, output, target):
      
      predicted = output.argmax(dim=-1)  # shape: (sequence_length, batch_size)
      equal_rows = 0
      for i in range(target.size(0)):
          # Remove the first and last indices and check if the sliced tensors are equal
          if torch.all(torch.eq(target[i, 1:-1], predicted[i, 1:-1])):
              equal_rows += 1
      matches=equal_rows

      # Compute the accuracy
      accuracy = matches / len(target) * 100
      return accuracy



In [19]:
hidden_layer_size=256
embedding_size= 64
cell_type="GRU"
drop_out=0.2
num_layers_encoders=3
num_layers_decoders=3
bidirectional=True
model = Seq2Seq(len(char_to_idx_latin)+2, len(char_to_idx_lang)+2, hidden_layer_size, embedding_size, cell_type,drop_out,num_layers_encoders,num_layers_decoders,bidirectional)
model.to(device)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(29, 64)
    (rnn): GRU(64, 256, num_layers=3, dropout=0.2)
  )
  (decoder): Decoder(
    (embedding): Embedding(256, 64)
    (rnn): GRU(64, 256, num_layers=3, dropout=0.2)
    (fc): Linear(in_features=256, out_features=67, bias=True)
  )
)

In [20]:
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name    | Type    | Params
------------------------------------
0 | encoder | Encoder | 1.0 M 
1 | decoder | Decoder | 1.1 M 
------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params
8.436     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
target=[[1,2],[1,3],[4,5]]
predicted=[[1,2],[1,3],[4,5]]

target = torch.tensor(target)
predicted = torch.tensor(predicted)
 
matches=0
for row in range(0,3):
  if(torch.sum(target[row]==predicted[row]) == 2):
    matches+=1
matches

In [ ]:

# Example tensors
a = torch.tensor([1, 2, 3, 4, 5])
b = torch.tensor([1, 6, 3, 4, 7])

# Remove the first and last elements of each tensor
a_sliced = a[1:-1]
b_sliced = b[1:-1]

a_sliced

In [ ]:
800/800 [00:39<00:00, 20.20it/s, v_num=10, train_accuracy_step=10.90, val_accuracy=19.70, train_accuracy_epoch=14.40]
